In [1]:
pip install duckdb

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
# import duckdb as dd

# # Create an in-memory DuckDB connection
# con = dd.connect(':memory:')

In [2]:
import duckdb as dd

# Create a persistent DuckDB database
con = dd.connect('my_database1.db')

In [3]:
import duckdb as db

# Create a relation from a SQL query
rel = db.sql("SELECT * FROM range(10_00) AS tbl(ID)")
# Display the relation
rel.show()

┌────────────┐
│     ID     │
│   int64    │
├────────────┤
│          0 │
│          1 │
│          2 │
│          3 │
│          4 │
│          5 │
│          6 │
│          7 │
│          8 │
│          9 │
│          · │
│          · │
│          · │
│        990 │
│        991 │
│        992 │
│        993 │
│        994 │
│        995 │
│        996 │
│        997 │
│        998 │
│        999 │
├────────────┤
│ 1000 rows  │
│ (20 shown) │
└────────────┘



In [4]:
con.sql('SHOW ALL TABLES')

┌──────────┬─────────┬─────────┬──────────────┬──────────────┬───────────┐
│ database │ schema  │  name   │ column_names │ column_types │ temporary │
│ varchar  │ varchar │ varchar │  varchar[]   │  varchar[]   │  boolean  │
├──────────┴─────────┴─────────┴──────────────┴──────────────┴───────────┤
│                                 0 rows                                 │
└────────────────────────────────────────────────────────────────────────┘

In [5]:
# Create second table
con.execute('''
CREATE OR REPLACE TABLE employees(
    id INTEGER,
    name VARCHAR,
    age INTEGER,
    salary DOUBLE
);
''')

# Insert some data in second table
con.execute('''
INSERT INTO employees VALUES
(1, 'Person 1', 30, 70000),
(2, 'Person 2', 25, 55000),
(3, 'Person 3', 35, 80000),
(4, 'Person 4', 45, 87000),
(5, 'Person 5', 43, 40000),
(6, 'Person 6', 65, 48000);
''')


# Result of showing tables after creating the second table
con.sql('SHOW ALL TABLES')

┌──────────────┬─────────┬───────────┬─────────────────────────┬─────────────────────────────────────┬───────────┐
│   database   │ schema  │   name    │      column_names       │            column_types             │ temporary │
│   varchar    │ varchar │  varchar  │        varchar[]        │              varchar[]              │  boolean  │
├──────────────┼─────────┼───────────┼─────────────────────────┼─────────────────────────────────────┼───────────┤
│ my_database1 │ main    │ employees │ [id, name, age, salary] │ [INTEGER, VARCHAR, INTEGER, DOUBLE] │ false     │
└──────────────┴─────────┴───────────┴─────────────────────────┴─────────────────────────────────────┴───────────┘

In [6]:
con.sql('SELECT * FROM employees;')

┌───────┬──────────┬───────┬─────────┐
│  id   │   name   │  age  │ salary  │
│ int32 │ varchar  │ int32 │ double  │
├───────┼──────────┼───────┼─────────┤
│     1 │ Person 1 │    30 │ 70000.0 │
│     2 │ Person 2 │    25 │ 55000.0 │
│     3 │ Person 3 │    35 │ 80000.0 │
│     4 │ Person 4 │    45 │ 87000.0 │
│     5 │ Person 5 │    43 │ 40000.0 │
│     6 │ Person 6 │    65 │ 48000.0 │
└───────┴──────────┴───────┴─────────┘

# Ingesting and Working with Data directly from files

In [7]:
con.sql('''
        INSERT INTO employees (id, name, age, salary) 
        (SELECT * FROM "employees.csv")
''')
con.sql('SELECT * FROM employees LIMIT 25;')

┌───────┬────────────┬───────┬──────────┐
│  id   │    name    │  age  │  salary  │
│ int32 │  varchar   │ int32 │  double  │
├───────┼────────────┼───────┼──────────┤
│     1 │ Person 1   │    30 │  70000.0 │
│     2 │ Person 2   │    25 │  55000.0 │
│     3 │ Person 3   │    35 │  80000.0 │
│     4 │ Person 4   │    45 │  87000.0 │
│     5 │ Person 5   │    43 │  40000.0 │
│     6 │ Person 6   │    65 │  48000.0 │
│    17 │ employee17 │    56 │ 100000.0 │
│    18 │ employee18 │    76 │ 125000.0 │
│    19 │ employee19 │    66 │  90000.0 │
│    20 │ employee20 │    45 │ 100000.0 │
├───────┴────────────┴───────┴──────────┤
│ 10 rows                     4 columns │
└───────────────────────────────────────┘

# Create country1 table and ingest data from countries1.csv

In [8]:
# Create a table
con.execute('''
CREATE OR REPLACE TABLE countries1(
    country VARCHAR,
    country_code1 VARCHAR,
    country_code2 VARCHAR,
    country_code INTEGER,
    region VARCHAR,
    sub_region VARCHAR,
    intermediate_region VARCHAR,
    region_code INTEGER,
    sub_region_code INTEGER,
    intermediate_region_code INTEGER

);
''')

# ingest data from csv file
con.sql('''
        INSERT INTO countries1 (country, country_code1, country_code2, country_code, region, sub_region, intermediate_region, region_code, sub_region_code, intermediate_region_code) 
        (SELECT * FROM "countries1.csv")
''')

# selec a few rows
con.sql('SELECT * FROM countries1 LIMIT 25;')

┌─────────────────────┬───────────────┬───────────────┬──────────────┬──────────┬─────────────────────────────────┬─────────────────────┬─────────────┬─────────────────┬──────────────────────────┐
│       country       │ country_code1 │ country_code2 │ country_code │  region  │           sub_region            │ intermediate_region │ region_code │ sub_region_code │ intermediate_region_code │
│       varchar       │    varchar    │    varchar    │    int32     │ varchar  │             varchar             │       varchar       │    int32    │      int32      │          int32           │
├─────────────────────┼───────────────┼───────────────┼──────────────┼──────────┼─────────────────────────────────┼─────────────────────┼─────────────┼─────────────────┼──────────────────────────┤
│ Afghanistan         │ AF            │ AFG           │            4 │ Asia     │ Southern Asia                   │ NULL                │         142 │              34 │                     NULL │
│ Åland Islands

# Create a table allcountries from all.csv

In [9]:
# Create a table
con.execute('''
CREATE OR REPLACE TABLE allcountries(
    country VARCHAR,
    country_code1 VARCHAR,
    country_code2 VARCHAR,
    country_code INTEGER,
    iso_3166_2 VARCHAR,
    region VARCHAR,
    sub_region VARCHAR,
    intermediate_region VARCHAR,
    region_code INTEGER,
    sub_region_code INTEGER,
    intermediate_region_code INTEGER

);
''')

# ingest data from a file

con.sql('''
        INSERT INTO allcountries (country, country_code1, country_code2, country_code, iso_3166_2, region, sub_region, intermediate_region, region_code, sub_region_code, intermediate_region_code) 
        (SELECT * FROM "all.csv")
''')

# select few rows
con.sql('SELECT * FROM allcountries LIMIT 25;')

┌─────────────────────┬───────────────┬───────────────┬──────────────┬───────────────┬──────────┬─────────────────────────────────┬─────────────────────┬─────────────┬─────────────────┬──────────────────────────┐
│       country       │ country_code1 │ country_code2 │ country_code │  iso_3166_2   │  region  │           sub_region            │ intermediate_region │ region_code │ sub_region_code │ intermediate_region_code │
│       varchar       │    varchar    │    varchar    │    int32     │    varchar    │ varchar  │             varchar             │       varchar       │    int32    │      int32      │          int32           │
├─────────────────────┼───────────────┼───────────────┼──────────────┼───────────────┼──────────┼─────────────────────────────────┼─────────────────────┼─────────────┼─────────────────┼──────────────────────────┤
│ Afghanistan         │ AF            │ AFG           │            4 │ ISO 3166-2:AF │ Asia     │ Southern Asia                   │ NULL            

# run queries

In [10]:
region_asia = con.sql('''
        SELECT  *
            FROM
                allcountries
            WHERE
                region = 'Africa'
''')

region_asia

┌────────────────────────────────┬───────────────┬───────────────┬──────────────┬───────────────┬─────────┬────────────────────┬─────────────────────┬─────────────┬─────────────────┬──────────────────────────┐
│            country             │ country_code1 │ country_code2 │ country_code │  iso_3166_2   │ region  │     sub_region     │ intermediate_region │ region_code │ sub_region_code │ intermediate_region_code │
│            varchar             │    varchar    │    varchar    │    int32     │    varchar    │ varchar │      varchar       │       varchar       │    int32    │      int32      │          int32           │
├────────────────────────────────┼───────────────┼───────────────┼──────────────┼───────────────┼─────────┼────────────────────┼─────────────────────┼─────────────┼─────────────────┼──────────────────────────┤
│ Algeria                        │ DZ            │ DZA           │           12 │ ISO 3166-2:DZ │ Africa  │ Northern Africa    │ NULL                │          

In [12]:
western_asia = con.sql('''
                            SELECT  *
                                FROM
                                    allcountries
                                WHERE
                                    region = 'Africa'
                                    AND sub_region = 'Northern Africa'
                    ''')

western_asia

┌────────────────┬───────────────┬───────────────┬──────────────┬───────────────┬─────────┬─────────────────┬─────────────────────┬─────────────┬─────────────────┬──────────────────────────┐
│    country     │ country_code1 │ country_code2 │ country_code │  iso_3166_2   │ region  │   sub_region    │ intermediate_region │ region_code │ sub_region_code │ intermediate_region_code │
│    varchar     │    varchar    │    varchar    │    int32     │    varchar    │ varchar │     varchar     │       varchar       │    int32    │      int32      │          int32           │
├────────────────┼───────────────┼───────────────┼──────────────┼───────────────┼─────────┼─────────────────┼─────────────────────┼─────────────┼─────────────────┼──────────────────────────┤
│ Algeria        │ DZ            │ DZA           │           12 │ ISO 3166-2:DZ │ Africa  │ Northern Africa │ NULL                │           2 │              15 │                     NULL │
│ Egypt          │ EG            │ EGY       

In [13]:
western_asia_ge = con.sql('''
                            SELECT  *
                                FROM
                                    allcountries
                                WHERE
                                    region = 'Africa'
                                    AND sub_region = 'Sub-Saharan Africa'
                                    AND intermediate_region = 'Eastern Africa'
                    ''')

western_asia_ge

┌────────────────────────────────┬───────────────┬───────────────┬──────────────┬───────────────┬─────────┬────────────────────┬─────────────────────┬─────────────┬─────────────────┬──────────────────────────┐
│            country             │ country_code1 │ country_code2 │ country_code │  iso_3166_2   │ region  │     sub_region     │ intermediate_region │ region_code │ sub_region_code │ intermediate_region_code │
│            varchar             │    varchar    │    varchar    │    int32     │    varchar    │ varchar │      varchar       │       varchar       │    int32    │      int32      │          int32           │
├────────────────────────────────┼───────────────┼───────────────┼──────────────┼───────────────┼─────────┼────────────────────┼─────────────────────┼─────────────┼─────────────────┼──────────────────────────┤
│ British Indian Ocean Territory │ IO            │ IOT           │           86 │ ISO 3166-2:IO │ Africa  │ Sub-Saharan Africa │ Eastern Africa      │          

# Working with Data directly from files
While DuckDB can ingest data from various formats, as discussed. DuckDB also gives a provision to read from these files into an in-memory DuckDB relation (table) and query them directly, to explore and work with data.



In [14]:
relation = con.sql('''
                        SELECT  *
                            FROM
                                'all.csv'
                            WHERE
                                region = 'Oceania'
                                AND "sub-region" = 'Polynesia'
                ''')
relation

┌───────────────────┬─────────┬─────────┬──────────────┬───────────────┬─────────┬────────────┬─────────────────────┬─────────────┬─────────────────┬──────────────────────────┐
│       name        │ alpha-2 │ alpha-3 │ country-code │  iso_3166-2   │ region  │ sub-region │ intermediate-region │ region-code │ sub-region-code │ intermediate-region-code │
│      varchar      │ varchar │ varchar │   varchar    │    varchar    │ varchar │  varchar   │       varchar       │   varchar   │     varchar     │         varchar          │
├───────────────────┼─────────┼─────────┼──────────────┼───────────────┼─────────┼────────────┼─────────────────────┼─────────────┼─────────────────┼──────────────────────────┤
│ American Samoa    │ AS      │ ASM     │ 016          │ ISO 3166-2:AS │ Oceania │ Polynesia  │ NULL                │ 009         │ 061             │ NULL                     │
│ Cook Islands      │ CK      │ COK     │ 184          │ ISO 3166-2:CK │ Oceania │ Polynesia  │ NULL               

In [22]:
type(relation)

duckdb.duckdb.DuckDBPyRelation

In [17]:
# These are known as DuckDB relation objects. We can display all data in these `relations`, as demonstrated above, or extract them as a list of tuples using the `fetchall` method of these relations.

all_relation = relation.fetchall()


In [18]:
type(all_relation)

list

In [19]:
type(all_relation[0])

tuple

In [20]:
all_relation[0]

('American Samoa',
 'AS',
 'ASM',
 '016',
 'ISO 3166-2:AS',
 'Oceania',
 'Polynesia',
 None,
 '009',
 '061',
 None)

# Integrating with Pandas
One of DuckDB’s most powerful features is its compatibility with `Pandas` (and other libraries like Polars etc.). You can run SQL queries directly on Pandas DataFrames or convert query results into DataFrames.



In [23]:
# Example: Converting to Pandas DataFrames

relation.to_df()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,American Samoa,AS,ASM,016,ISO 3166-2:AS,Oceania,Polynesia,None,009,061,None
1,Cook Islands,CK,COK,184,ISO 3166-2:CK,Oceania,Polynesia,None,009,061,None
2,French Polynesia,PF,PYF,258,ISO 3166-2:PF,Oceania,Polynesia,None,009,061,None
3,Niue,NU,NIU,570,ISO 3166-2:NU,Oceania,Polynesia,None,009,061,None
4,Pitcairn,PN,PCN,612,ISO 3166-2:PN,Oceania,Polynesia,None,009,061,None
5,Samoa,WS,WSM,882,ISO 3166-2:WS,Oceania,Polynesia,None,009,061,None
6,Tokelau,TK,TKL,772,ISO 3166-2:TK,Oceania,Polynesia,None,009,061,None
7,Tonga,TO,TON,776,ISO 3166-2:TO,Oceania,Polynesia,None,009,061,None
8,Tuvalu,TV,TUV,798,ISO 3166-2:TV,Oceania,Polynesia,None,009,061,None
9,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,None,009,061,None


In [24]:
# Example: Querying Pandas DataFrames directly

import pandas as pd

df = pd.DataFrame({
    'id': [14, 15, 16],
    'name': ['Person 14', 'Person 15', 'Person 16'],
    'age': [45, 40, 35],
    'salary': [100000, 85000, 75000]
})

con.sql('''
        INSERT INTO employees (id, name, age, salary)
        SELECT * FROM df
''')

con.sql('select * from employees')

┌───────┬────────────┬───────┬──────────┐
│  id   │    name    │  age  │  salary  │
│ int32 │  varchar   │ int32 │  double  │
├───────┼────────────┼───────┼──────────┤
│     1 │ Person 1   │    30 │  70000.0 │
│     2 │ Person 2   │    25 │  55000.0 │
│     3 │ Person 3   │    35 │  80000.0 │
│     4 │ Person 4   │    45 │  87000.0 │
│     5 │ Person 5   │    43 │  40000.0 │
│     6 │ Person 6   │    65 │  48000.0 │
│    17 │ employee17 │    56 │ 100000.0 │
│    18 │ employee18 │    76 │ 125000.0 │
│    19 │ employee19 │    66 │  90000.0 │
│    20 │ employee20 │    45 │ 100000.0 │
│    14 │ Person 14  │    45 │ 100000.0 │
│    15 │ Person 15  │    40 │  85000.0 │
│    16 │ Person 16  │    35 │  75000.0 │
├───────┴────────────┴───────┴──────────┤
│ 13 rows                     4 columns │
└───────────────────────────────────────┘

In [25]:
con.sql('SELECT * FROM df').df()

,id,name,age,salary
0,14,Person 14,45,100000
1,15,Person 15,40,85000
2,16,Person 16,35,75000


# Working with Parquet and Arrow
DuckDB also supports efficient handling of Parquet and Arrow formats, commonly used in big data scenarios. You can read data from Parquet files and run SQL queries on them without first loading them into memory.



In [ ]:
# Example: Reading from Parquet Files

# Read data from a Parquet file
con.sql("SELECT * FROM 'countries.parquet'").df().head()

In [ ]:
# Similarly, DuckDB integrates well with Apache Arrow and supports operations on Arrow tables.

# Advanced Queries and Using DuckDB for Analytical Workloads
# DuckDB is optimized for performance, especially for analytical queries. DuckDB’s architecture, particularly its use of vectorized execution and columnar storage, helps DuckDB to speed up query processing and make it extremely efficient for data analytics. Additionally, DuckDB can operate directly on compressed data formats like Parquet, reducing the need for data decompression.

# Window Functions: You can perform windowing operations (e.g., running totals, and moving averages, etc.).
# Group By: Complex group-by operations with large datasets are optimized.
# Parallel Execution: DuckDB automatically parallelizes many operations for faster results on large datasets.

In [28]:
con.sql("show tables")

┌──────────────┐
│     name     │
│   varchar    │
├──────────────┤
│ allcountries │
│ countries1   │
│ employees    │
└──────────────┘

In [29]:
# Example: Group and count countries by their regions.

con.sql('''
        SELECT  region
                , COUNT(DISTINCT country) AS country_counts
            FROM
                allcountries
            GROUP BY
                region
            ORDER BY
                country_counts DESC
''')

┌──────────┬────────────────┐
│  region  │ country_counts │
│ varchar  │     int64      │
├──────────┼────────────────┤
│ Africa   │             60 │
│ Americas │             57 │
│ Europe   │             51 │
│ Asia     │             50 │
│ Oceania  │             29 │
│ NULL     │              2 │
└──────────┴────────────────┘

In [30]:
# Example: Group and count countries by their regions.

con.sql('''
        SELECT  region
                , COUNT(DISTINCT country) AS country_counts
            FROM
                countries1
            GROUP BY
                region
            ORDER BY
                country_counts DESC
''')

┌──────────┬────────────────┐
│  region  │ country_counts │
│ varchar  │     int64      │
├──────────┼────────────────┤
│ Africa   │             60 │
│ Americas │             57 │
│ Europe   │             51 │
│ Asia     │             50 │
│ Oceania  │             29 │
│ NULL     │              2 │
└──────────┴────────────────┘

In [31]:
# Example: Calculate the average salary and find people with above average salary.

# Writing CTE's
con.sql('''
        WITH avg_salary AS (
            SELECT
                    ROUND(AVG(salary),2) AS avg_salary
                FROM
                    employees
            )
        
        SELECT
                *
            FROM
                employees
            WHERE
                salary > (SELECT avg_salary FROM avg_salary)
        
''')

┌───────┬────────────┬───────┬──────────┐
│  id   │    name    │  age  │  salary  │
│ int32 │  varchar   │ int32 │  double  │
├───────┼────────────┼───────┼──────────┤
│     4 │ Person 4   │    45 │  87000.0 │
│    17 │ employee17 │    56 │ 100000.0 │
│    18 │ employee18 │    76 │ 125000.0 │
│    19 │ employee19 │    66 │  90000.0 │
│    20 │ employee20 │    45 │ 100000.0 │
│    14 │ Person 14  │    45 │ 100000.0 │
│    15 │ Person 15  │    40 │  85000.0 │
└───────┴────────────┴───────┴──────────┘

In [32]:
# Closing the Connection:

con.close()